In [ ]:
%matplotlib widget
from bmcs_shell.api import WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBTessellation4P, WBCell4Param
import numpy as np
import matplotlib.pyplot as plt
import gmsh
import pygmsh
import k3d

In [ ]:
params = valid_params_2_cells_span_1800_height_300_width_500= {'a': 100.0, 'b': 493.47789764898204, 'c': 188.08268555670807, 'gamma': 0.9231728219305352, 'n_phi_plus': 3}

wbt4p = WBTessellation4P(
                        **params,
                         n_x_plus=2,
                         wireframe_width=5,
                         trim_half_cells_along_y=True,
                         trim_half_cells_along_x=True,
                         align_outer_nodes_along_x=True,
)
wbt4p.interact()

In [ ]:

tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=1
n_x_plus=1
wbm = WBShellAnalysis(h=10, tmodel=tmodel)
data = dict(gamma=0.33, a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=False)
wbm.geo.trait_set(**data)

In [ ]:
wbm.xdomain.mesh.direct_mesh = False
wbm.xdomain.mesh.subdivision = 2

In [ ]:
mesh = wbm.xdomain.mesh.mesh
mesh

In [ ]:
X_Ia = wbt4p.X_Ia_trimmed
X_Ia

In [ ]:
I_Fi = wbt4p.I_Fi_trimmed
I_Fi

In [ ]:
wb = WBCell4Param(gamma = 1)
wb.interact()

In [ ]:
X_Ia = wb.X_Ia / 2000
I_Fi = wb.I_Fi
X_Ia

### Flipping negative normals to unite all normals point (upwards)

In [ ]:
X_Fia = X_Ia[I_Fi]
To_flip_F = np.cross(X_Fia[:, 1, :] - X_Fia[:, 0, :], X_Fia[:, 2, :] - X_Fia[:, 0, :])[:, 2] < 0 # all items where z of normal is negative
I_Fi[To_flip_F] = np.flip(I_Fi[To_flip_F], axis=1)
I_Fi

In [ ]:
np.sin(np.pi/4)

In [ ]:
np.sqrt(2)/2

In [ ]:
X_Ia = np.copy(X_Ia)
I_Fi = np.copy(I_Fi)

# Flip normals to have same direction
X_Fia = X_Ia[I_Fi]
normals_Fi = np.cross(X_Fia[:, 1, :] - X_Fia[:, 0, :], X_Fia[:, 2, :] - X_Fia[:, 0, :])
To_flip_F = normals_Fi[:, 2] < 0  # all items where z of normal is negative
I_Fi[To_flip_F] = np.flip(I_Fi[To_flip_F], axis=1)
# Update X_Fia
X_Fia = X_Ia[I_Fi]
normals_Fi = np.cross(X_Fia[:, 1, :] - X_Fia[:, 0, :], X_Fia[:, 2, :] - X_Fia[:, 0, :])
normals_Fi_norm = normals_Fi / np.sqrt(np.sum(normals_Fi*normals_Fi, axis=1))[:, np.newaxis]
normals_Fi_norm

In [ ]:
# See https://gitlab.onelab.info/gmsh/gmsh/-/blob/master/tutorials/python/t11.py
# dimTag means tuple of (dimention, tag). Where tag is like an ID

import gmsh
import sys
import numpy as np
import bmcs_utils.api as bu
from scipy.spatial.transform import Rotation

thickness = 10
# thickness = 10 * normals_Fi_norm

# Each dimension Dim has Tags refering to objects starting from 1, 
#  (a point is dim=1, a line, surface or more is dim=2 and a volume is dim=3)

gmsh.initialize()
gmsh.model.add("wb_extrude")
mesh_size = 0.1

# Adding outer area of the pattern with extrusion: ------------------------------------------------

xpoints = np.array([gmsh.model.occ.addPoint(*X_a, mesh_size) for X_a in X_Ia])

wb_facets = []

for I_i in I_Fi:
    xpoints1 = xpoints[I_i]
    curves = [gmsh.model.occ.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.occ.addLine(xpoints1[-1], xpoints1[0])]

    cl = gmsh.model.occ.addCurveLoop(curves)
    pl = gmsh.model.occ.addPlaneSurface([cl])
    wb_facets.append(pl)
    
# #     To generate quadrangles instead of triangles, we can simply add
#     gmsh.model.mesh.setRecombine(1, pl)

# pg = gmsh.model.addPhysicalGroup(dim = 3, tags=wb_facets, name='pg')
# print(pg)

# Extrude (extrude is already a volume or CAD object)
for i, wb_facet in enumerate(wb_facets):
#     ext = gmsh.model.occ.extrude(dimTags=[(2, wb_facet)], dx=thickness[i, 0], dy=thickness[i, 1], 
#                                  dz=thickness[i, 2], numElements=[], heights=[], recombine=True)
    ext = gmsh.model.occ.extrude(dimTags=[(2, wb_facet)], dx=0, dy=0, 
                             dz=1, numElements=[], heights=[], recombine=True)

vols = gmsh.model.occ.getEntities(dim=3)
# tess_block = gmsh.model.occ.fuse(vols[len(creases):], vols[len(creases):])
tess_block = gmsh.model.occ.fuse(vols, vols)

print('vols=', gmsh.model.occ.getEntities(dim=3))
print('vols.size=', len(gmsh.model.occ.getEntities(dim=3)))

vols = gmsh.model.occ.getEntities(dim=3)

# gmsh.model.occ.cut([vols[1]], [vols[0]])

# gmsh.model.occ.remove(dimTags=gmsh.model.occ.getEntities(dim=2), recursive=True)

# max3DTag = gmsh.model.occ.getMaxTag(3)

# Meshing ---------------------------------------------------- ------------------------------------------------

gmsh.model.occ.synchronize()

# field = gmsh.model.mesh.field
# field.add("MathEval", 1)
# field.setString(1, "F", "1")
# field.setAsBackgroundMesh(1)

# # To generate quadrangles instead of triangles, we can simply add
# gmsh.model.mesh.setRecombine(2, pl)

# If we'd had several surfaces, we could have used the global option
# "Mesh.RecombineAll":
#
# gmsh.option.setNumber("Mesh.RecombineAll", 1)

# You can also set the subdivision step alone, with
#
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

# gmsh.model.mesh.generate(2)

# Note that you could also apply the recombination algorithm and/or the
# subdivision step explicitly after meshing, as follows:
#
print('before generate')
# gmsh.model.mesh.generate(2)
print('mesh generated')
# gmsh.model.mesh.recombine()
print('mesh recombine')
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
# gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()

In [ ]:
import gmsh
import sys
import numpy as np

gmsh.initialize()

gmsh.model.add("t11")

# We have seen in tutorials `t3.py' and `t6.py' that extruded and transfinite
# meshes can be "recombined" into quads, prisms or hexahedra. Unstructured
# meshes can be recombined in the same way. Let's define a simple geometry with
# an analytical mesh size field:

X_Id = np.array([[0, 0, 0], 
                [1, 0, 0], 
                [1, 1, 0],
                 [1, 1, 1]])
# I_Fi = np.array([[0, 1, 2], [0, 1, 3]])
I_Fi = np.array([[0, 1, 2]])
mesh_size = 0.5


# geom.extrude()
xpoints = np.array([gmsh.model.geo.addPoint(*X_d) for X_d in X_Id])
print(xpoints)

for I_i in I_Fi:
    xpoints1 = xpoints[I_i]
    curves = [gmsh.model.geo.addLine(xpoints1[k], xpoints1[k + 1]) for k in range(len(xpoints1) - 1)] + [gmsh.model.geo.addLine(xpoints1[-1], xpoints1[0])]

    cl = gmsh.model.geo.addCurveLoop(curves)
    pl = gmsh.model.geo.addPlaneSurface([cl])
    
    print('pl=', pl)
    
    # To generate quadrangles instead of triangles, we can simply add
#     gmsh.model.mesh.setRecombine(1, pl)
pg = gmsh.model.addPhysicalGroup(3, tags=[1, 2])
gmsh.model.geo.extrude(dimTags=[(2, pg)], dx=0, dy=0, dz=0.1, numElements=[1], heights=[], recombine=False)
# gmsh.model.geo.extrude(dimTags=[(2, 1)], dx=0, dy=0, dz=0.1, numElements=[1], heights=[], recombine=True)
# gmsh.model.geo.extrude(dimTags=[(2, 2)], dx=0, dy=0.1, dz=-0.1, numElements=[3], heights=[], recombine=True)

gmsh.model.geo.synchronize()

# field = gmsh.model.mesh.field
# field.add("MathEval", 1)
# field.setString(1, "F", "1")
# field.setAsBackgroundMesh(1)

# # To generate quadrangles instead of triangles, we can simply add
# gmsh.model.mesh.setRecombine(2, pl)

# If we'd had several surfaces, we could have used the global option
# "Mesh.RecombineAll":
#
# gmsh.option.setNumber("Mesh.RecombineAll", 1)

# The default recombination algorithm is called "Blossom": it uses a minimum
# cost perfect matching algorithm to generate fully quadrilateral meshes from
# triangulations. More details about the algorithm can be found in the
# following paper: J.-F. Remacle, J. Lambrechts, B. Seny, E. Marchandise,
# A. Johnen and C. Geuzaine, "Blossom-Quad: a non-uniform quadrilateral mesh
# generator using a minimum cost perfect matching algorithm", International
# Journal for Numerical Methods in Engineering 89, pp. 1102-1119, 2012.

# For even better 2D (planar) quadrilateral meshes, you can try the
# experimental "Frontal-Delaunay for quads" meshing algorithm, which is a
# triangulation algorithm that enables to create right triangles almost
# everywhere: J.-F. Remacle, F. Henrotte, T. Carrier-Baudouin, E. Bechet,
# E. Marchandise, C. Geuzaine and T. Mouton. A frontal Delaunay quad mesh
# generator using the L^inf norm. International Journal for Numerical Methods
# in Engineering, 94, pp. 494-512, 2013. Uncomment the following line to try
# the Frontal-Delaunay algorithms for quads:
#
# gmsh.option.setNumber("Mesh.Algorithm", 8)

# The default recombination algorithm might leave some triangles in the mesh, if
# recombining all the triangles leads to badly shaped quads. In such cases, to
# generate full-quad meshes, you can either subdivide the resulting hybrid mesh
# (with `Mesh.SubdivisionAlgorithm' set to 1), or use the full-quad
# recombination algorithm, which will automatically perform a coarser mesh
# followed by recombination, smoothing and subdivision. Uncomment the following
# line to try the full-quad algorithm:
#
# gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2) # or 3

# You can also set the subdivision step alone, with
#
# gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

gmsh.model.mesh.generate(2)

# Note that you could also apply the recombination algorithm and/or the
# subdivision step explicitly after meshing, as follows:
#
# gmsh.model.mesh.generate(2)
gmsh.model.mesh.recombine()
gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
gmsh.model.mesh.refine()

# Launch the GUI to see the results:
if '-nopopup' not in sys.argv:
    gmsh.fltk.run()

gmsh.finalize()


In [ ]:
X_Id = np.array([[0, 0, 0], 
                [1, 0, 0], 
                [1, 1, 0],
                 [1, 0, 1]])
I_Fi = np.array([[0, 1, 2], [0, 1, 3]])
mesh_size = 0.5

with pygmsh.geo.Geometry() as geom:
    # geom.extrude()
    xpoints = np.array([
        geom.add_point(X_d, mesh_size=mesh_size) for X_d in X_Id
    ])
    print(xpoints)
    surfaces = []
    for I_i in I_Fi:
        xpoints1 = xpoints[I_i]
        curves = [
                          geom.add_line(xpoints1[k], xpoints1[k + 1])
                          for k in range(len(xpoints1) - 1)
                      ] + [geom.add_line(xpoints1[-1], xpoints1[0])]

        curve_loop = geom.add_curve_loop(curves)
        # self.surface = geom.add_plane_surface(ll, holes) if make_surface else None
        surfaces.append(geom.add_plane_surface(curve_loop))
        
#         self.dim_tag = self.surface.dim_tag
#         self.dim_tags = self.surface.dim_tags
#         self._id = self.surface._id

    sl1 = geom.add_surface_loop(surfaces)
#     v1 = geom.add_volume([sl1])
#     ext = geom.extrude(sl1, (0, 0.1, 0.1))
#     ext = geom.extrude(surface, (0, 0, 0.1))

    #                geom.add_polygon(X_id, mesh_size=mesh_size)
    # gmsh.model.geo.remove_all_duplicates()
    
    # To create quads + tri instead of tri
    geom.set_recombined_surfaces(surfaces)
    
    mesh1 = geom.generate_mesh()
ext

In [ ]:
mesh1.points

In [ ]:
mesh1.cells[2][1]

In [ ]:
mesh1.points

In [ ]:
mesh1.cells[2][1][:, [0, 3]]

In [ ]:
plot = k3d.plot()

for l in mesh1.cells[2][1][:, [0, 1]]:
    mesh = k3d.line(mesh1.points[l], dtype=np.float_)
    plot += mesh
for l in mesh1.cells[2][1][:, [1, 2]]:
    mesh = k3d.line(mesh1.points[l], dtype=np.float_)
    plot += mesh
for l in mesh1.cells[2][1][:, [2, 3]]:
    mesh = k3d.line(mesh1.points[l], dtype=np.float_)
    plot += mesh
mesh = k3d.mesh(mesh1.points, mesh1.cells[1][1], wireframe=True)
plot += mesh
plot

In [ ]:
plot = k3d.plot()
mesh = k3d.mesh(mesh1.points, mesh1.cells[1][1], wireframe=True)
plot += mesh
plot